# 使用一等函数实现设计模式

## 案例分析: 重构”策略“模式

**”策略“模式：**
> 定义一系列算法，把他们一一封装起来，并且使它们可以相互替换。本模式使得算法可以独立于使用它的客户而变化。

### 经典模式

In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # 上下文
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
            
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): # 策略:抽象基类
    
    @abstractmethod
    def discount(self, order):
        """返回折扣金额(正值)"""
    
class FidelityPromo(Promotion): # 第一个具体策略
    """为积分为1000或以上的顾客，提供5%折扣"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
    
class BulkItemPromo(Promotion): # 第二个具体策略
    """单个商品为20个或者以上时，可以提供10%折扣"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion):
    """订单中的不同商品达到10个或者以上时，提供7%折扣"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [3]:
# 示例
joe = Customer('John Joe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())


<Order total: 10.00 due: 9.30>

In [8]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

### 使用函数实现”策略“模式



In [13]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # 上下文
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
            
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

    
def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
            
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [14]:
joe = Customer('John Joe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [15]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [16]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

上述代码表示，没必要在新建订单时实例化新的促销对象，函数拿来即用。

### 选择最佳策略： 简单的方式

如下定义比较容易，不过如果要添加策略，就比较麻烦

In [17]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [19]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

### 找出模块中的全部策略

globals()

返回一个字典，表示当前的全局符号表。这个符号表之中针对当前模块（对函数或方法来说，是指定义它们的模块，而不是调用它们的模块）


In [20]:
promos = [globals()[name] for name in globals()
         if name.endswith('_promo')
         and name != 'best_promo']

def best_promo(order):
    return max(promo(order) for promo in promos)

## "命令"模式

